# Textgrid durations

> "Get max/min segment durations from textgrids"

- branch: master
- comments: false
- categories: [textgrid, hsi]

In [7]:
from praatio import textgrid

In [2]:
input = "/Users/joregan/Desktop/"

In [8]:
from pathlib import Path

input_path = Path(input)
IS_DIR = False
if input_path.is_dir():
    IS_DIR = True
assert input_path.exists()

In [29]:
import re

def get_durations(filename):
    tg = textgrid.openTextgrid(filename, includeEmptyIntervals=False)

    if len(tg.tierNames) == 1:
        tier = tg.getTier(tg.tierNames[0])
    elif "whisperx" in tg.tierNames:
        tier = tg.getTier("whisperx")
    elif "utterances" in tg.tierNames:
        tier = tg.getTier("utterances")
    elif "words" in tg.tierNames:
        tier = tg.getTier("words")
    else:
        print("Be careful: file", filename, "has none of the expected tier names")

    longest_utt = 0.0
    longest_sil = 0.0
    shortest_utt = 100.0
    shortest_sil = 100.0

    def silence_like(text):
        text = text.strip()

        if text == "":
            return True
        if text in ["sp", "sil", "<eps>"]:
            return True
        m = re.match("^\[[^]]+\]$", text)
        if m:
            return True
        return False        

    silences = []
    utterances = []

    for interval in tier.entries:
        start = interval[0]
        end = interval[1]
        text = interval[2]
        duration = end - start

        text = text.strip()
        if silence_like(text):
            silences.append(duration)
            if duration > longest_sil:
                longest_sil = duration
            if duration < shortest_sil:
                shortest_sil = duration
        else:
            utterances.append(duration)
            if duration > longest_utt:
                longest_utt = duration
            if duration < shortest_utt:
                shortest_utt = duration
    
    return {
        "longest_utt": longest_utt,
        "longest_sil": longest_sil,
        "shortest_utt": shortest_utt,
        "shortest_sil": shortest_sil,
        "utterances": utterances,
        "silences": silences
    }
            

In [36]:
import statistics

longest_utt = 0.0
longest_sil = 0.0
shortest_utt = 100.0
shortest_sil = 100.0

silences = []
utterances = []

if IS_DIR:
    for filename in input_path.glob("*.[Tt]ext[Gg]rid"):
        res = get_durations(str(filename))
        silences += res["silences"]
        utterances += res["utterances"]
        longest_utt = max(longest_utt, res["longest_utt"])
        longest_sil = max(longest_sil, res["longest_sil"])
        shortest_utt = min(shortest_utt, res["shortest_utt"])
        shortest_sil = min(shortest_sil, res["shortest_sil"])
else:
    res = get_durations(str(input_path))
    silences = res["silences"]
    utterances = res["utterances"]
    longest_utt = res["longest_utt"]
    longest_sil = res["longest_sil"]
    shortest_utt = res["shortest_utt"]
    shortest_sil = res["shortest_sil"]

print(f"Longest utterance: {longest_utt:.02f}")
print(f"Shortest utterance: {shortest_utt:.02f}")
print(f"Longest silence: {longest_sil:.02f}")
print(f"Shortest silence: {shortest_sil:.02f}")

print(f"Average utterance: {statistics.fmean(utterances):.02f}")
print(f"Average silence: {statistics.fmean(silences):.02f}")

Longest utterance: 27.59
Shortest utterance: 0.20
Longest silence: 2.04
Shortest silence: 0.13
Average utterance: 2.54
Average silence: 0.64
